<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/sl_whisper_medium_test_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-h4pbppxp
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-h4pbppxp
  Resolved https://github.com/huggingface/transformers to commit 2c658b5a4282f2e824b4e23dc3bcda7ef27d5827
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8119970 sha256=0cc30485dfcb5817e2226f77497fada440f31fce68c5217495d37661db14e28d
  Stored in directory: /tmp/pip-ephem-wheel-cache-cyg9f7kz/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [3]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [72]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "sl", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [73]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 309
})

In [74]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

In [75]:
model = whisper.load_model("medium")

In [76]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [77]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [78]:
text

,0
0,seveda lahko čestitamo za hiter razvoj elektro...
1,anžetu logarju in njegovim staršem je s smrtjo...
2,gre pa za to da imajo priložnost biti aktivni ...
3,zato je pomembno da evropska zveza kot sila mi...
4,vlado republike slovenije pozivam naj čim prej...
...,...
304,ta rešitev o kateri govorimo pa bo žal nezados...
305,sem za ukrepanje na področju podnebnih spremem...
306,sama ji nisem zelo naklonjena ker jo lahko raz...
307,beneška komisija je bila pri svojih zapisih ze...


In [79]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [80]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,male,seveda lahko čestitamo za hiter razvoj elektro...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.004638671875, 0.000244140625, -0.0036010742...",16000
1,male,anžetu logarju in njegovim staršem je s smrtjo...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0118408203125, -0.013702392578125, -0.0115...",16000
2,male,gre pa za to da imajo priložnost biti aktivni ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.002410888671875, 0.006072998046875, 0.00354...",16000
3,male,zato je pomembno da evropska zveza kot sila mi...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.02227783203125, -0.00787353515625, 0.02120...",16000
4,male,vlado republike slovenije pozivam naj čim prej...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00048828125, -0.0001220703125, -0.001007080...",16000
...,...,...,...,...,...
304,female,ta rešitev o kateri govorimo pa bo žal nezados...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000335693359375, -0.00262451171875, 0.01065...",16000
305,female,sem za ukrepanje na področju podnebnih spremem...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0211181640625, -0.003387451171875, 0.00769...",16000
306,female,sama ji nisem zelo naklonjena ker jo lahko raz...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00250244140625, -0.001983642578125, -0.002...",16000
307,male,beneška komisija je bila pri svojih zapisih ze...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001373291015625, 9.1552734375e-05, -0.00161...",16000


In [81]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
0,male,seveda lahko čestitamo za hiter razvoj elektro...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.004638671875, 0.000244140625, -0.0036010742...",16000
1,male,anžetu logarju in njegovim staršem je s smrtjo...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0118408203125, -0.013702392578125, -0.0115...",16000
2,male,gre pa za to da imajo priložnost biti aktivni ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.002410888671875, 0.006072998046875, 0.00354...",16000
3,male,zato je pomembno da evropska zveza kot sila mi...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.02227783203125, -0.00787353515625, 0.02120...",16000
4,male,vlado republike slovenije pozivam naj čim prej...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00048828125, -0.0001220703125, -0.001007080...",16000
...,...,...,...,...,...
297,male,nekateri seveda postavljajo tukaj dilemo ali n...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.1724853515625, -0.188323974609375, -0.1896...",16000
302,male,sistem vračanja iz grčije v turčijo ne deluje ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00018310546875, 0.000946044921875, 9.155273...",16000
303,male,od komisije pričakujemo da bo predložila smise...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0035400390625, -0.00372314453125, -0.00039...",16000
307,male,beneška komisija je bila pri svojih zapisih ze...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001373291015625, 9.1552734375e-05, -0.00161...",16000


In [82]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
7,female,na nekaj pa bi rada opozorila tukaj v svojem g...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000244140625, -0.003326416015625, -0.004821...",16000
12,female,iz tega razloga je tudi napačno sklepati da la...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.044647216796875, -0.03350830078125, -0.019...",16000
18,female,gre namreč za ugled nas vseh gre za ugled evro...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.03363037109375, -0.033935546875, -0.036132...",16000
20,female,z vsem spoštovanjem do novega organa in do izj...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.1650390625, 0.13531494140625, 0.11520385742...",16000
26,female,v naši energetski mešanici je vse več premoga.,/root/.cache/huggingface/datasets/downloads/ex...,"[0.003143310546875, 0.00390625, 6.103515625e-0...",16000
...,...,...,...,...,...
300,female,moram reči da sem po dolgem času zadovoljna z ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.004669189453125, 0.012481689453125, 0.00991...",16000
301,female,napačna je namreč razlaga da je kategorija ope...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.024261474609375, 0.028076171875, 0.03970336...",16000
304,female,ta rešitev o kateri govorimo pa bo žal nezados...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000335693359375, -0.00262451171875, 0.01065...",16000
305,female,sem za ukrepanje na področju podnebnih spremem...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0211181640625, -0.003387451171875, 0.00769...",16000


In [83]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="sl")
    return result["text"]

In [84]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [85]:
total_wer = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.4090909090909091
Processed 2 files, WER: 0.46153846153846156
Processed 3 files, WER: 0.26666666666666666
Processed 4 files, WER: 0.75
Processed 5 files, WER: 0.40625
Processed 6 files, WER: 0.43902439024390244
Processed 7 files, WER: 1.0
Processed 8 files, WER: 0.6666666666666666
Processed 9 files, WER: 0.3076923076923077
Processed 10 files, WER: 0.2
Processed 11 files, WER: 1.0
Processed 12 files, WER: 0.4090909090909091
Processed 13 files, WER: 0.43478260869565216
Processed 14 files, WER: 0.5806451612903226
Processed 15 files, WER: 1.0
Processed 16 files, WER: 0.7142857142857143
Processed 17 files, WER: 0.38235294117647056
Processed 18 files, WER: 0.3333333333333333
Processed 19 files, WER: 0.68
Processed 20 files, WER: 0.7307692307692307
Processed 21 files, WER: 0.6190476190476191
Processed 22 files, WER: 0.6451612903225806
Processed 23 files, WER: 0.9473684210526315
Processed 24 files, WER: 0.5454545454545454
Processed 25 files, WER: 1.0
Processed 26 files

In [86]:
total_wer = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.654320987654321
Processed 2 files, WER: 1.1333333333333333
Processed 3 files, WER: 0.36363636363636365
Processed 4 files, WER: 0.5
Processed 5 files, WER: 0.125
Processed 6 files, WER: 0.23809523809523808
Processed 7 files, WER: 0.21428571428571427
Processed 8 files, WER: 0.48
Processed 9 files, WER: 0.5263157894736842
Processed 10 files, WER: 0.45454545454545453
Processed 11 files, WER: 0.25
Processed 12 files, WER: 0.8235294117647058
Processed 13 files, WER: 0.23809523809523808
Processed 14 files, WER: 0.4
Processed 15 files, WER: 0.3333333333333333
Processed 16 files, WER: 0.4444444444444444
Processed 17 files, WER: 0.45
Processed 18 files, WER: 0.39285714285714285
Processed 19 files, WER: 0.23529411764705882
Processed 20 files, WER: 0.35294117647058826
Processed 21 files, WER: 0.2857142857142857
Processed 22 files, WER: 0.2727272727272727
Processed 23 files, WER: 0.3157894736842105
Processed 24 files, WER: 0.32
Processed 25 files, WER: 0.3181818181818182
P